<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/datacollection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1. 저장된 fullcode 목록을 불러와서 종목별 데이터 수집
2. pickle 저장

(프로젝트를 시작할 때 1번만 실행)
'''

In [1]:
import json
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/k-project/output/'
data_path = '/content/drive/MyDrive/[Shared]K-Project/'
# pickle 경로를 설정하기 위해

Mounted at /content/drive


In [2]:
# 한국시간 설정
import pytz

KST = pytz.timezone('Asia/Seoul')
today = str(datetime.now(KST).strftime("%Y%m%d"))
print(today)

20211001


In [3]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
fullcode

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [8]:
#상장폐지 상태 제외시키기.
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']
fullcode_filter

,full_code,short_code,codeName,marketEngName,status
0,KR7060310000,060310,3S,KOSDAQ,상장
1,KR7095570008,095570,AJ네트웍스,KOSPI,상장
2,KR7006840003,006840,AK홀딩스,KOSPI,상장
3,KR7054620000,054620,APS홀딩스,KOSDAQ,상장
4,KR7265520007,265520,AP시스템,KOSDAQ,상장
...,...,...,...,...,...
2580,KR7000542001,000547,흥국화재2우B,KOSPI,상장
2581,KR7000541003,000545,흥국화재우,KOSPI,상장
2582,KR7003280005,003280,흥아해운,KOSPI,상장
2583,KR7037440005,037440,희림,KOSDAQ,상장


In [9]:
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
# fullcode열을 list로 변경 후 확인

['KR7060310000', 'KR7095570008', 'KR7006840003', 'KR7054620000', 'KR7265520007', 'KR7211270004', 'KR7027410000', 'KR7282330000', 'KR7138930003', 'KR7001460005']


In [ ]:
'''
종가 시가 저가 고가 시가총액 EPS PER BPS PBR 주당배당금 배당수익률
(* 필요한 컬럼)

df2
 0   TRD_DD : 거래일자 *
 1   ISU_CD : 종목코드 *      
 2   ISU_NM : 종목명 *      
 3   MKT_NM : 시장코드 (kospi)      
 4   SECUGRP_NM : 증권구분 (주권)    
 5   TDD_CLSPRC : 종가 *  
 6   FLUC_TP_CD    
 7   CMPPRVDD_PRC : 대비 
 8   FLUC_RT : 등락률      
 9   TDD_OPNPRC : 시가 *  
 10  TDD_HGPRC  : 고가 *  
 11  TDD_LWPRC  : 저가 *  
 12  ACC_TRDVOL : 거래량   
 13  ACC_TRDVAL : 거래대금   
 14  MKTCAP : 시가총액 *

df3
 0   TRD_DD : 거래일자        
 1   TDD_CLSPRC : 종가    
 2   FLUC_TP_CD     
 3   CMPPREVDD_PRC : 대비 
 4   FLUC_RT : 등락률      
 5   EPS *           
 6   PER *           
 7   BPS *           
 8   PBR *           
 9   DPS : 주당배당금 *          
 10  DVD_YLD : 배당수익률 *
 '''

In [ ]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

strtDd = '19000101'
endDd = today

for i in fullcode_list[:2]: # 테스트를 위해 2개만 실행 (두 종목만 출력됨)
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': i,
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2=pd.DataFrame(j2['output'])
    # print(df2)
    # print('------------------------------------------------')

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': i,
    'isuCd2': i,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3=pd.DataFrame(j2['output'])
    # print(df3)
    # print('------------------------------------------------')

    df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    df3_ = df3[['TRD_DD', 'EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]

    result = pd.merge(df2_, df3_)
    print(result)
    print('------------------------------------------------')

          TRD_DD  ISU_CD  ISU_NM TDD_CLSPRC TDD_OPNPRC TDD_HGPRC TDD_LWPRC  \
0     2021/10/01  060310      3S      3,960      4,080     4,080     3,925   
1     2021/09/30  060310      3S      4,105      4,225     4,270     4,010   
2     2021/09/29  060310      3S      4,160      3,865     4,180     3,805   
3     2021/09/28  060310      3S      3,985      4,070     4,140     3,950   
4     2021/09/27  060310      3S      4,115      3,840     4,140     3,750   
...          ...     ...     ...        ...        ...       ...       ...   
4804  2002/04/29  060310  삼에스코리아      6,780      6,800     7,200     6,780   
4805  2002/04/26  060310  삼에스코리아      7,700      8,550     8,750     7,700   
4806  2002/04/25  060310  삼에스코리아      8,750      9,530     9,530     7,490   
4807  2002/04/24  060310  삼에스코리아      8,510      8,370     8,510     7,750   
4808  2002/04/23  060310  삼에스코리아      7,600      7,600     7,600     7,600   

               MKTCAP  EPS     PER  BPS   PBR DPS DVD_YLD  
0  

In [ ]:
## Save pickle
# with open(output_path + "df2.pickle","wb") as fw:
#     pickle.dump(df2, fw)